# Normal 01b - THM concentration in tap water - Stan version

# _Josep Fortiana_   $\hskip5cm$      2022-03-21

****
###### LaTeX macros
$\def\argmax{\operatorname{arg\,max}}$
$\def\argmin{\operatorname{arg\,min}}$
$\def\binomdist{\operatorname{Binomial}}$
$\def\berndist{\operatorname{Bernoulli}}$
$\def\betabinomdist{\operatorname{Beta-Binomial}}$
$\def\betadist{\operatorname{Beta}}$
$\def\betahat{\kern1.3pt\hat{\kern-1.3pt \beta}}$
$\def\borel{\operatorname{Borel}}$
$\def\cauchydist{\operatorname{Cauchy}}$
$\def\CondTo{\mathbin{|\mskip0.5mu}}$
$\def\cdf{\operatorname{cdf}}$
$\def\cov{\operatorname{cov}}$
$\def\cor{\operatorname{cor}}$
$\def\dexpdist{\operatorname{DExp}}$
$\def\diag{\operatorname{diag}}$
$\def\dprod{\displaystyle\prod}$
$\def\dsum{\displaystyle\sum}$
$\def\eqdef{\stackrel{\text{def}}{\mskip3mu=\mskip3mu}}$
$\def\cE{\cal E}$
$\def\cP{\cal P}$
$\def\cZ{\cal Z}$
$\def\Ee{\operatorname{E}}$
$\def\expdist{\operatorname{Exponential}}$
$\def\fdist{\operatorname{F}}$
$\def\gammadist{\operatorname{Gamma}}$
$\def\geomdist{\operatorname{Geom}}$
$\def\halfcauchydist{\operatorname{Cauchy}^{+}}$
$\def\hyperdist{\operatorname{Hypergeom}}$
$\def\invgammadist{\operatorname{InvGamma}}$
$\def\invwishartdist{\operatorname{InvWishart}}$
$\def\hypergeomdist{\operatorname{Hypergeom}}$
$\def\indep{\perp{\mskip-16mu}\perp}$
$\DeclareMathOperator{\indica}{\mathbb{1}}$
$\DeclareMathOperator{\invgammadist}{InvGamma}$
$\newcommand{\KL}[2]{\operatorname{KL}(\mkern2mu#1\mkern2mu\mathrel\mid\mathrel{\mkern-8mu}\mid\mkern2mu#2\mkern2mu)}$
$\def\laplacedist{\operatorname{Laplace}}$
$\def\lognormaldist{\operatorname{Log-Normal}}$
$\def\median{\operatorname{median}}$
$\DeclareMathOperator{\multinomialdist}{Multinomial}$
$\DeclareMathOperator{\multinomdist}{Multinomial}$
$\def\N{\mathbb{N}}$
$\def\negbindist{\operatorname{NegativeBinomial}}$
$\def\normaldist{\operatorname{Normal}}$
$\def\Normaldist{\operatorname{MNormal}}$
$\def\pdf{\operatorname{pdf}}$
$\def\pmf{\operatorname{pmf}}$
$\def\poissondist{\operatorname{Poisson}}$
$\def\prob{\operatorname{P}}$
$\def\R{\mathbb{R}}$ 
$\def\stilde{\tilde{s}}$ 
$\def\sqtilde{\tilde{s}{}^{2}}$ 
$\def\tdist{\operatorname{t}}$
$\def\u{\mathbb{1}}$
$\def\unifdist{\operatorname{Unif}}$
$\def\va{\text{v.a.}}$
$\def\var{\operatorname{var}}$
$\def\Var{\operatorname{Var}}$
$\def\xbar{\kern1.3pt\overline{\kern-1.3pt x}}$
$\def\Xbar{\kern1.3pt\overline{\kern-1.3pt X}}$
$\def\yhat{\kern1.3pt\hat{\kern-1.3pt y}}$
$\def\Z{\mathbb{Z}}$
***

# Case study: THM concentrations ($\sigma$ known)

This problem appears in the [Web course _Bayesian Statistics for Epidemiologists_](http://www.columbia.edu/~cjd11/charles_dimaggio/DIRE/styled-4/styled-11/code-4/), also in Lunn _et al._ (2012) _The BUGS book,_ pp. 43-45.

### Description of the problem

Local UK public health agencies test water samples for trihalomethanes (THM) in every supply zone a couple of times each year. 

We want to learn about the average concentration in a particular zone based on $n=2$ measurements, $x_{1}$ and $x_{2}$, with mean $\overline{x}=130$ and known $\sigma=5$. 

What is the best estimate of the mean concentration $\theta$?

In [ ]:
n<-2
xbar<-130
sigma<-5

## Specify model

$$
     \begin{array}{lcll}
     x_{i}  &\sim &\normaldist(\theta,\sigma^{2}),\mskip30mu 1\leq i\leq n,&\mskip60mu\text{Observed data,}\\[0.3cm]
     \theta &\sim &\normaldist(\mu,\gamma^{2}),                              &\mskip60mu\text{Prior.}
          \end{array}
$$

# Treatment with Stan

In [ ]:
#install.packages("rstan", dependencies=TRUE,repos= "https://cloud.r-project.org")
require(rstan)
# Following directions:
# For execution on a local, multicore CPU with excess RAM we recommend calling
# options(mc.cores = parallel::detectCores()).
# To avoid recompilation of unchanged Stan programs, we recommend calling
# rstan_options(auto_write = TRUE)
options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)

## 1. To obtain the prior predictive pdf, before observing  $\,\overline{x}$

In [ ]:
modelString.0 <- "data {
    real mu;                 // Normal prior mean
    real<lower=0> gamma;     // Normal prior sd
    real<lower=0> sigma;    //  sd of observed INDIVIDUAL observation
    }
parameters {
    real theta;          // parameter is an unbounded real (mean of observed xbar)
    }
transformed parameters {
    }
model {
    // prior
    theta ~ normal(mu,gamma); 

    // no likelihood here
   
    }
generated quantities {
    real x0;
    
// prior prediction
    x0 = normal_rng(theta,sigma);
    }
"

In [ ]:
# Translate model to C++ and compile to DSO:
stanDso.0 <- stan_model(model_code=modelString.0) 

In [ ]:
# Data:
mu<-120
gamma<-10
sigma<-5
dataList.0<-list(mu=mu,gamma=gamma,sigma=sigma)

In [ ]:
# Generate sample of the prior predictive pdf:
stanFit.0<-sampling(object=stanDso.0,data = dataList.0, 
                     chains = 3,
                     iter = 5000, 
                     warmup = 600, 
                     thin = 1)

In [ ]:
S<-summary(stanFit.0)
round(S$summary,4)

### Compare with theoretical results

The prior predictive distribution should be:

$$
   x\sim\operatorname{N}\big(\mu,\sigma^{2}+\gamma^{2}\big)
$$

In [ ]:
Theor.mean.prior.pred<-mu
Theor.sd.prior.pred<-sqrt(sigma^2+gamma^2)
round(Theor.mean.prior.pred,4)
round(Theor.sd.prior.pred,4)

In [ ]:
# extract x0 sample, discard unnecessary attributes to keep just the numbers
x0<-extract(stanFit.0,'x0')
x0<-unlist(x0,use.names=FALSE)
str(x0)

In [ ]:
StanSim.mean.prior.pred<-mean(x0)
StanSim.sd.prior.pred<-sd(x0)
round(StanSim.mean.prior.pred,4)
round(StanSim.sd.prior.pred,4)

In [ ]:
# plot simulated data
options(repr.plot.width=7,repr.plot.height=7)
x.min<-min(x0)
x.max<-max(x0)
hist(x0,breaks=35,freq=FALSE,col="LightGreen",xlim=c(x.min,x.max),ylim=c(0,0.038),
     xlab="x",main="Stan-simulated prior predictive histogram")
# Superimpose the theoretical density
x<-seq(x.min,x.max,length=1000)
lines(x,dnorm(x,mean=Theor.mean.prior.pred,sd=Theor.sd.prior.pred),lwd=2.5,col="DarkGreen")

## Histogram by the default Stan function

In [ ]:
options(repr.plot.width=6,repr.plot.height=6)
stan_hist(stanFit.0,pars=c("theta"),binwidth=4)

## Credible interval plot

In [ ]:
options(repr.plot.width=6,repr.plot.height=3)
SP<-stan_plot(stanFit.0,pars=c("theta"))
str(SP$data)  # Print bounds of 80% and 95% credible intervals
SP

## Density plot

In [ ]:
options(repr.plot.width=6,repr.plot.height=6)
stan_dens(stanFit.0,pars=c("theta"))

## Diagnostics: traceplot, autocorrelation plot

In [ ]:
options(repr.plot.width=10,repr.plot.height=6)
stan_trace(stanFit.0,pars=c("theta"))

In [ ]:
options(repr.plot.width=10,repr.plot.height=6)
stan_ac(stanFit.0,pars=c("theta"))

## 2. Posterior pdf and posterior predictive pdf

### Now we include likelihood and  $\,\overline{x}$

This posterior predictive distribution can be used to compute the probability that the water company will be fined next semester, which will happen if next month measurement (predicted)  $\tilde{𝑥}$  exceeds  $x_{\text{crit}}=145$.

We include an indicator `Ind_crit` in the `generated quantities` block.

In [ ]:
modelString.2 <- "data {
    real mu;                 // Normal prior mean
    real<lower=0> gamma;     // Normal prior sd
    int<lower=0> n;         //  sample size
    real xbar;              //  Observed mean
    real<lower=0> sigma;    // sd(x), assumed known
    real<lower=0> sigma_n;   // This is the sd of the mean, equal to sd(x)/sqrt(n)
    real x_crit;            // Critical value, to assess the probability of a fine
    }
parameters {
    real theta;          
    }
transformed parameters {
    }
model {
    // prior
    theta ~ normal(mu,gamma); 

    //likelihood
    xbar ~ normal(theta,sigma_n);
    }
generated quantities {
    real x_pred;
    real<lower=0,upper=1> Ind_crit;     // An indicator to detect x_pred>x_crit

// posterior prediction  (here Stan will sample theta from the POSTERIOR pdf)
    x_pred = normal_rng(theta,sigma);   // here variance must be sigma, NOT sigma_n
                                        // we are predicting a *single* new observation
                                        
    Ind_crit= int_step(x_pred-x_crit);
    }
"

In [ ]:
# Translate model to C++ and compile to DSO:
stanDso.2 <- stan_model(model_code=modelString.2) 

In [ ]:
# Data:
mu<-120
gamma<-10
sigma<-5
n<-2
sigma_n<-sigma/sqrt(n)
xbar<-130
x_crit<-145
dataList.2<-list(mu=mu,gamma=gamma,sigma=sigma,sigma_n=sigma_n,xbar=xbar,x_crit=x_crit)

In [ ]:
# Generate sample of the prior predictive pdf:
stanFit.2<-sampling(object=stanDso.2,data = dataList.2, 
                     chains = 3,
                     iter = 15000, 
                     warmup = 3500, 
                     thin = 1)

In [ ]:
S<-summary(stanFit.2)
round(S$summary,4)

## Posterior predictive distribution

The posterior predictive distribution for a new observation $\widetilde{x}$, given the observed $\overline{x}$, is normal, with mean $\mu_{x}$ and variance the sum $\sigma^{2}+\tau^{2}$.

In [ ]:
alpha<-(sigma^2/n)/((sigma^2/n)+gamma^2)
mux<-alpha*mu+(1-alpha)*xbar
tau2<-((sigma^2/n)*gamma^2)/((sigma^2/n)+gamma^2)
tau<-sqrt(tau2)
round(mux,4)
round(tau2,4)
round(tau,4)

In [ ]:
Theor.m.post.pred<-mux
Theor.var.post.pred<-sigma^2+tau^2
Theor.sd.post.pred<-sqrt(Theor.var.post.pred)
round(Theor.m.post.pred,3)
round(Theor.var.post.pred,3)
round(Theor.sd.post.pred,3)

In [ ]:
round(Theor.m.post.pred,3)
round(Theor.var.post.pred,3)
round(Theor.sd.post.pred,3)

In [ ]:
# extract sample from stanFit object, discarding unnecessary attributes
x_pred<-extract(stanFit.2,'x_pred')
x_pred<-unlist(x_pred,use.names=FALSE)
str(x_pred)

In [ ]:
StanSim.mean.post.pred<-mean(x_pred)
StanSim.var.post.pred<-var(x_pred)
StanSim.sd.post.pred<-sd(x_pred)
round(StanSim.mean.post.pred,4)
round(StanSim.var.post.pred,4)
round(StanSim.sd.post.pred,4)

## Predicting next semester probability of fine

In [ ]:
Sim.p.fine.1<-sum(x_pred>x_crit)/length(x_pred)
round(Sim.p.fine.1,4)

In [ ]:
# With Ind_crit should be the same result, shouldn't it?
Ind_crit <- extract(stanFit.2, 'Ind_crit')
Ind_crit <- unlist(Ind_crit, use.names=FALSE)
str(Ind_crit)

In [ ]:
Sim.p.fine<-sum(Ind_crit)/length(Ind_crit)
round(Sim.p.fine,4)
# Theoretical value was 0.003669567

## Histogram by the default Stan function

In [ ]:
options(repr.plot.width=12,repr.plot.height=6)
stan_hist(stanFit.2,pars=c("theta","x_pred"),binwidth=2)

## Credible interval plot

In [ ]:
options(repr.plot.width=8,repr.plot.height=4)
SP<-stan_plot(stanFit.2,pars=c("theta","x_pred"))
str(SP$data)  # Print bounds of 80% and 95% credible intervals
SP

## Density plot

In [ ]:
options(repr.plot.width=12,repr.plot.height=6)
stan_dens(stanFit.2,pars=c("theta","x_pred"))

## Diagnostics: traceplot, autocorrelation plot

In [ ]:
options(repr.plot.width=14,repr.plot.height=6)
stan_trace(stanFit.2,pars=c("theta","x_pred"))

In [ ]:
options(repr.plot.width=15,repr.plot.height=6)
stan_ac(stanFit.2,pars=c("theta","x_pred"))

Here we can add or replace the previous plots by. e.g. those generated with `bayesplot`

# Same diagrams with the `bayesplot` package

In [ ]:
#install.packages("bayesplot", dependencies=TRUE,repos= "https://cloud.r-project.org")
require(bayesplot)

In [ ]:
color_scheme_set("green")
options(repr.plot.width=10,repr.plot.height=6)
mcmc_trace(stanFit.2, pars = c("theta"))

In [ ]:
color_scheme_set("viridis")
options(repr.plot.width=10,repr.plot.height=6)
mcmc_trace(stanFit.2, pars = c("theta"))

In [ ]:
options(repr.plot.width=10,repr.plot.height=6)
mcmc_trace_highlight(stanFit.2, pars = c("theta"), highlight = 3)

In [ ]:
options(repr.plot.width=10,repr.plot.height=6)
mcmc_acf(stanFit.2,pars=c("theta"))

In [ ]:
color_scheme_set("viridisA")
options(repr.plot.width=10,repr.plot.height=6)
mcmc_acf_bar(stanFit.2,pars=c("theta"))

## Analysis of posterior pdf properties

### Posterior credible interval 

In [ ]:
color_scheme_set("purple")
options(repr.plot.width=8,repr.plot.height=5)
mcmc_intervals(stanFit.2, pars = c("theta"))
# Defaults are
# prob = 0.5,
# prob_outer = 0.9,

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
mcmc_areas(stanFit.2, pars = c("theta"),prob=0.75,prob_outer=0.995)

In [ ]:
color_scheme_set("viridisA")
options(repr.plot.width=7,repr.plot.height=7)
mcmc_hist(stanFit.2, pars = c("theta"),binwidth=0.025)

In [ ]:
options(repr.plot.width=7,repr.plot.height=7)
mcmc_dens(stanFit.2, pars = c("theta"))

In [ ]:
color_scheme_set("viridisD")
options(repr.plot.width=7,repr.plot.height=7)
mcmc_dens_chains(stanFit.2, pars = c("theta"))

In [ ]:
color_scheme_set("blue")
options(repr.plot.width=7,repr.plot.height=7)
mcmc_violin(stanFit.2, pars = c("theta"))